In [2]:
!export PATH=/home/adamprice/miniconda3/envs/u75a/lib${PATH:+:${PATH}}

In [1]:
!export LD_LIBRARY_PATH=/home/adamprice/miniconda3/envs/u75a/lib:${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}

In [10]:
!pip install tensorflow==2.2.*

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.5/516.5 MB 1.8 MB/s eta 0:00:0000:0100:05
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Using cached numpy-1.18.5-cp38-cp38-manylinux1_x86_64.whl (20.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 3.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.6/454.6 kB 3.1 MB/s eta 0:00:00a 0:00:01
  Using cached google_auth-1.35.0-py2.py3-none-any.whl (152 kB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.10.0
    Uninstalling tensorflow-estimator-2.10.0:
      Successfully uninstalled tensorflow-estimator-2.10.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: gast
    Found existing installation: gast 0.2.2
    Uninstalling gast-0.2.2:
      Successfully uninstalled gast-0.2.2
  Attempting uninstall: google-auth

In [3]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
import pandas as pd
from keras.models import Model
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Input
#from ProcessTrueStateActionData import read_df_in_chunks

2023-03-30 12:46:24.889206: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-30 12:46:25.092970: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-30 12:46:25.901276: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/adamprice/miniconda3/envs/u75a/lib/
2023-03-30 12:46:25.901384: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_

### Load and preprocess the data 
(produce tf train+test datasets)

In [5]:
train_test_split = 0.75
data_path = '/home/adamprice/u75a-Data-Efficient-Decisions/CybORG/CybORG/Notebooks/logs/PPO/no_decoy_200000'
nodes = np.load(data_path + '/data/nodes.npy')
actions = np.load(data_path + '/data/actions.npy')
node_id = np.load(data_path + '/data/node_id.npy')
next_nodes = np.load(data_path + '/data/next_nodes.npy')
exploit = np.load(data_path + '/data/exploit.npy')
scan = np.load(data_path + '/data/scan.npy')
privileged = np.load(data_path + '/data/privileged.npy')
user = np.load(data_path + '/data/user.npy')
unknown = np.load(data_path + '/data/unknown.npy')
no = np.load(data_path + '/data/no.npy')


In [6]:
data = np.concatenate([node_id, nodes], axis=1)

In [1]:
!pip install tensoRT

ERROR: Could not find a version that satisfies the requirement tensoRT (from versions: none)
ERROR: No matching distribution found for tensoRT


In [7]:
STATE_LEN = 91
ACTION_LEN = 3
    
max_train_epochs = 50

losses = []
input_ = Input(shape=(data.shape[1],))
x = Dense(128, activation='relu', name='hidden')(input_)
outs = []

outs.append(Dense(3, activation='softmax', name='activity')(x))
outs.append(Dense(4, activation='softmax', name='compromised')(x))
losses.append(tf.keras.losses.CategoricalCrossentropy())

def scheduler(epoch, lr):
    if epoch < 2:
        return lr
    else:
        return lr * tf.math.exp(-0.15)

base_model = Model(input_, outs)
base_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=losses, metrics=[tf.keras.metrics.CategoricalAccuracy()])
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, min_delta=0.0005)
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)


: 

: 

In [ ]:
data_map = {}
data_map['activity'] = next_nodes[p,:3]
data_map['compromised'] = next_nodes[p,3:]
with tf.device("/device:GPU:1"):
    history = base_model.fit(data[p,:], data_map, epochs=max_train_epochs, validation_split=0.25, 
                                    verbose=0, callbacks=[es_callback, lr_callback], batch_size=512, shuffle=True)